## Spark Getting Started

### University of California, Santa Barbara  
### PSTAT 135/235: Big Data Analytics
### Last Updated: January 29, 2019

---  


Source: Learning Spark

Chapter 1: Introduction to Data Analysis with Spark  
Chapter 2: Getting Started

### OBJECTIVES
-  Spark background
-  Setup and installation
-  Basic concepts
-  Minimal code examples
-  Running Spark: Interactive Session
-  Running Spark: Command Line

### CONCEPTS

- Functional programming

- SparkSession - single point of entry to interact w Spark functionality

- SparkContext - used as entry points, from Spark 1.0

- Resilient Distributed Datasets (RDDs) - Spark’s fundamental abstraction for distributed data and computation

- Dataset

- Driver Program - contains application main function, defines RDDs on cluster, applies operations to them.

- Worker Node or Executor - the units that perform tasks

---

*Spark Background*

- Designed to be fast  
no waiting around for hours, need to work interactively with data  


- Designed to handle big data


- General Purpose  
Unlike Hadoop, several modules in one place, such as machine learning, batch, queries, streaming


- Caching is possible, so intermediate data can be stored in memory on workers


- Highly accessible:simple APIs to Python, Java, Scala, R, SQL  
Integrates w other Big Data tools such as Hadoop, Cassandra  
Can access HDFS data, Amazon S3, and others


**Documentation from README**  
You can find the latest Spark documentation, including a programming
guide, on the [project web page](http://spark.apache.org/documentation.html).

For general development tips, including info on developing Spark using an IDE, see   
[http://spark.apache.org/developer-tools.html](the Useful Developer Tools page).

Spark also comes with several sample programs in the `examples` directory.  
To run one of them in a shell, use `./bin/run-example <class> [params]`.  

For example:

In [ ]:
./bin/run-example SparkPi

will run the Pi example locally.

**Install**  

Page 9 provides step-by-step download and install instructions

Depends:
1. Python needs to be installed
2. Java 6 or higher needs to be installed

Change logging level (Page 12), change from: 

log4j.rootCategory=INFO  

To  

log4j.rootCategory=WARN  

# Interactive Python shell

From installed location and using ($) to denote prompt:


In [ ]:
$ bin\pyspark

Set up a minimal case Spark Session:
- using the local machine as master
- naming the app

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .master("local") \
        .appName("pspark_test") \
        .getOrCreate()

In [2]:
# print info about the session
spark

In [4]:
sc = spark.sparkContext


### RDDs and Datasets

Before Spark 2.0, the main programming interface of Spark was the *Resilient Distributed Dataset (RDD)*.  

After Spark 2.0, RDDs are replaced by *Dataset*, which is strongly-typed like an RDD, but with richer optimizations under the hood. 

The RDD interface is still supported  

Using Dataset is recommended, and it has better performance than RDD.

## Computing

### Example 1: Read lines from text file

In [5]:
import os

In [6]:
data_path = '/home/jovyan/UCSB_BigDataAnalytics/data/'

In [7]:
data_filename = 'README.md'

In [8]:
lines = spark.read.text(os.path.join(data_path, data_filename))

In [9]:
lines.count()

103

In [10]:
lines.first()

Row(value='# Apache Spark')

In [11]:
lines.collect()

[Row(value='# Apache Spark'),
 Row(value=''),
 Row(value='Spark is a fast and general cluster computing system for Big Data. It provides'),
 Row(value='high-level APIs in Scala, Java, Python, and R, and an optimized engine that'),
 Row(value='supports general computation graphs for data analysis. It also supports a'),
 Row(value='rich set of higher-level tools including Spark SQL for SQL and DataFrames,'),
 Row(value='MLlib for machine learning, GraphX for graph processing,'),
 Row(value='and Spark Streaming for stream processing.'),
 Row(value=''),
 Row(value='<http://spark.apache.org/>'),
 Row(value=''),
 Row(value=''),
 Row(value='## Online Documentation'),
 Row(value=''),
 Row(value='You can find the latest Spark documentation, including a programming'),
 Row(value='guide, on the [project web page](http://spark.apache.org/documentation.html).'),
 Row(value='This README file only contains basic setup instructions.'),
 Row(value=''),
 Row(value='## Building Spark'),
 Row(value=''),
 

In [12]:
type(lines.collect())

list

In [15]:
type(lines.collect()[0])

pyspark.sql.types.Row

### Example 2: Text Search  - print all lines containing “Spark”

In [21]:
spark_lines = lines.filter(lines.value.contains("Spark"))

In [22]:
# return list of first 5 records
spark_lines.take(5)   

[Row(value='# Apache Spark'),
 Row(value='Spark is a fast and general cluster computing system for Big Data. It provides'),
 Row(value='rich set of higher-level tools including Spark SQL for SQL and DataFrames,'),
 Row(value='and Spark Streaming for stream processing.'),
 Row(value='You can find the latest Spark documentation, including a programming')]

In [23]:
type(spark_lines)

pyspark.sql.dataframe.DataFrame

### Example 3: Word Count

In [25]:
# Read the file into an RDD
lines = sc.textFile(os.path.join(data_path, data_filename))

In [26]:
type(lines)

pyspark.rdd.RDD

In [27]:
words = lines.flatMap(lambda x: x.split())

In [28]:
words.take(5)

['#', 'Apache', 'Spark', 'Spark', 'is']

In [29]:
wordcounts = words.map(lambda x: (x, 1)) \
                  .reduceByKey(lambda x,y:x+y) \
                  .map(lambda x:(x[1],x[0])) \
                  .sortByKey(False)

In [33]:
wordcounts = words.map(lambda x: (x, 1))
wordcounts.take(5)

[('#', 1), ('Apache', 1), ('Spark', 1), ('Spark', 1), ('is', 1)]

In [38]:
wordcounts = words.map(lambda x: (x, 1)) \
                  .reduceByKey(lambda x,y:x+y)
wordcounts.take(5)

[('#', 1), ('Apache', 1), ('Spark', 16), ('is', 6), ('a', 8)]

In [ ]:
wordcounts = words.map(lambda x: (x, 1)) \
                  .reduceByKey(lambda x,y:x+y) \
                  .map(lambda x:(x[1],x[0]))
wordcounts.take(5)

In [30]:
wordcounts.take(10)

[(24, 'the'),
 (17, 'to'),
 (16, 'Spark'),
 (12, 'for'),
 (9, 'and'),
 (9, '##'),
 (8, 'a'),
 (7, 'can'),
 (7, 'on'),
 (7, 'run')]